In [1]:
!gcc --version

gcc (conda-forge gcc 14.1.0-1) 14.1.0
Copyright (C) 2024 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
!which gcc

/scratch/user/shipalnomoo/.conda/envs/jl/bin/gcc


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import stan
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import cm
from scipy import stats
from scipy.stats import pearsonr
import pickle

FONT_SIZE = 8

plt.rc('font', size=FONT_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=FONT_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=FONT_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=FONT_SIZE)    # legend fontsize
plt.rc('figure', titlesize=FONT_SIZE)  # fontsize of the figure title
plt.rcParams['pdf.fonttype'] = 42

Need to change the default location to cache. On HPRC, home directory only has 10 GB.

In [4]:
import os
os.environ['XDG_CACHE_HOME'] = '/scratch/user/shipalnomoo/PIP_CACHE/'

In [5]:
os.environ['XDG_CACHE_HOME']

'/scratch/user/shipalnomoo/PIP_CACHE/'

In [6]:
# from IPython import get_ipython
# get_ipython().run_line_magic('matplotlib', 'qt')
# plt.style.use('seaborn-white')

#%%

### load data ###
dat_RC = pd.read_csv('smFISH_RC.csv')

jtime_RC = dat_RC['Time'].values
AreaNormed_RC = dat_RC['AreaNormed'].values
CountsNr1d1_RC = dat_RC['Counts Nr1d1'].values
CountsCry1_RC = dat_RC['Counts Cry1'].values

dat_BC = pd.read_csv('smFISH_BC.csv')

jtime_BC = dat_BC['Time'].values
AreaNormed_BC = dat_BC['AreaNormed'].values
CountsBmal1_BC = dat_BC['Counts Bmal1'].values
CountsCry1_BC = dat_BC['Counts Cry1'].values

CountsCry1TOT = np.concatenate((CountsCry1_BC,CountsCry1_RC))
jtime_Cry1TOT = np.concatenate((jtime_BC,jtime_RC))
AreaCry1TOT = np.concatenate((AreaNormed_BC,AreaNormed_RC))

w = 2 * np.pi / 24 

TimevecRC = np.array([21.,25.,29.,33.,37.,41.]).reshape(-1,1)
TimevecBC = np.array([17.,21.,25.,29.,33.,37.,41.]).reshape(-1,1)

In [7]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
asyncio.run(asyncio.sleep(1))

In [8]:
#%%
# load Fourier parameters
Nr1d1_params, Cry1_params, Bmal1_params = pickle.load(open('FourierParams.pkl', 'rb'))

In [9]:
#%%

# MODEL 1

model1 = """
data{
    int<lower=1> N1;
    array[N1] int CountsNr1d1_RC;
    array[N1] int CountsCry1_RC; 
    
    array[N1] real jtime_RC;
    array[N1] real AreaNormed_RC;
    int<lower=1> N2;
    array[N2] int CountsBmal1_BC;
    array[N2] int CountsCry1_BC;
    array[N2] real jtime_BC;
    array[N2] real AreaNormed_BC;
    int <lower=1> N_f;
    array[N_f] real Nr1d1_params;
    array[N_f] real Cry1_params;
    array[N_f] real Bmal1_params;
    real w;
}
parameters{
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1;
        b2 = burstNr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1;
        b4 = burstBmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    freq_scaleCry1~ normal(0,100);
    freq_scaleNr1d1~ normal(0,100);
    freq_scaleBmal1~ normal(0,100);
    burstCry1~ normal(0,100);
    burstNr1d1~ normal(0,100);
    burstBmal1~ normal(0,100);
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1;
        b2 = burstNr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1;
        b4 = burstBmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}
 

"""

#%%


dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [14]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model1, data=dat, random_seed=194838)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/job.11303346/httpstan_fa7c6xvo/model_fzarby6w.stan', line 74, column 25: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_fa7c6xvo/model_fzarby6w.stan', line 73, column 25: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_fa7c6xvo/model_fzarby6w.stan', line 72, column 24: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_fa7c6xvo/model_fzarby6w.stan', line 71, column 30: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in 

In [18]:
# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=48, num_samples=2000, delta=0.95)

Sampling:   0%
Sampling:   0% (200/144000)
Sampling:   0% (400/144000)
Sampling:   0% (700/144000)
Sampling:   1% (1000/144000)
Sampling:   1% (1300/144000)
Sampling:   1% (1600/144000)
Sampling:   1% (1900/144000)
Sampling:   2% (2300/144000)
Sampling:   2% (2600/144000)
Sampling:   2% (3000/144000)
Sampling:   2% (3300/144000)
Sampling:   3% (3700/144000)
Sampling:   3% (4100/144000)
Sampling:   3% (4400/144000)
Sampling:   3% (4800/144000)
Sampling:   4% (5200/144000)
Sampling:   4% (5600/144000)
Sampling:   4% (6000/144000)
Sampling:   4% (6400/144000)
Sampling:   5% (6800/144000)
Sampling:   5% (7200/144000)
Sampling:   5% (7600/144000)
Sampling:   6% (8000/144000)
Sampling:   6% (8500/144000)
Sampling:   6% (8900/144000)
Sampling:   7% (9400/144000)
Sampling:   7% (9900/144000)
Sampling:   7% (10300/144000)
Sampling:   8% (10800/144000)
Sampling:   8% (11300/144000)
Sampling:   8% (11800/144000)
Sampling:   9% (12300/144000)
Sampling:   9% (12700/144000)
Sampling:   9% (13100/144

Sampling:  47% (67200/144000)
Sampling:  47% (67400/144000)
Sampling:  47% (67600/144000)
Sampling:  47% (67800/144000)
Sampling:  47% (68000/144000)
Sampling:  47% (68200/144000)
Sampling:  47% (68300/144000)
Sampling:  48% (68500/144000)
Sampling:  48% (68700/144000)
Sampling:  48% (68800/144000)
Sampling:  48% (69000/144000)
Sampling:  48% (69100/144000)
Sampling:  48% (69200/144000)
Sampling:  48% (69300/144000)
Sampling:  48% (69400/144000)
Sampling:  48% (69500/144000)
Sampling:  48% (69600/144000)
Sampling:  48% (69700/144000)
Sampling:  48% (69800/144000)
Sampling:  49% (69900/144000)
Sampling:  49% (70000/144000)
Sampling:  49% (70100/144000)
Sampling:  49% (70200/144000)
Sampling:  49% (70300/144000)
Sampling:  49% (70500/144000)
Sampling:  49% (70600/144000)
Sampling:  49% (70800/144000)
Sampling:  49% (70900/144000)
Sampling:  49% (71000/144000)
Sampling:  49% (71100/144000)
Sampling:  49% (71200/144000)
Sampling:  50% (71400/144000)
Sampling:  50% (71600/144000)
Sampling: 

Sampling:  69% (99800/144000)
Sampling:  69% (99900/144000)
Sampling:  70% (100100/144000)
Sampling:  70% (100300/144000)
Sampling:  70% (100500/144000)
Sampling:  70% (100700/144000)
Sampling:  70% (100900/144000)
Sampling:  70% (101100/144000)
Sampling:  70% (101300/144000)
Sampling:  70% (101500/144000)
Sampling:  71% (101700/144000)
Sampling:  71% (101900/144000)
Sampling:  71% (102000/144000)
Sampling:  71% (102100/144000)
Sampling:  71% (102300/144000)
Sampling:  71% (102400/144000)
Sampling:  71% (102500/144000)
Sampling:  71% (102600/144000)
Sampling:  71% (102700/144000)
Sampling:  71% (102800/144000)
Sampling:  71% (102900/144000)
Sampling:  72% (103000/144000)
Sampling:  72% (103100/144000)
Sampling:  72% (103200/144000)
Sampling:  72% (103300/144000)
Sampling:  72% (103400/144000)
Sampling:  72% (103500/144000)
Sampling:  72% (103600/144000)
Sampling:  72% (103700/144000)
Sampling:  72% (103800/144000)
Sampling:  72% (103900/144000)
Sampling:  72% (104000/144000)
Sampling: 

Sampling:  91% (131300/144000)
Sampling:  91% (131400/144000)
Sampling:  91% (131500/144000)
Sampling:  91% (131600/144000)
Sampling:  91% (131700/144000)
Sampling:  92% (131800/144000)
Sampling:  92% (131900/144000)
Sampling:  92% (132000/144000)
Sampling:  92% (132100/144000)
Sampling:  92% (132200/144000)
Sampling:  92% (132300/144000)
Sampling:  92% (132400/144000)
Sampling:  92% (132600/144000)
Sampling:  92% (132800/144000)
Sampling:  92% (132900/144000)
Sampling:  92% (133100/144000)
Sampling:  92% (133200/144000)
Sampling:  93% (133400/144000)
Sampling:  93% (133500/144000)
Sampling:  93% (133600/144000)
Sampling:  93% (133700/144000)
Sampling:  93% (133800/144000)
Sampling:  93% (134000/144000)
Sampling:  93% (134200/144000)
Sampling:  93% (134400/144000)
Sampling:  93% (134600/144000)
Sampling:  94% (134800/144000)
Sampling:  94% (135000/144000)
Sampling:  94% (135200/144000)
Sampling:  94% (135400/144000)
Sampling:  94% (135600/144000)
Sampling:  94% (135700/144000)
Sampling

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp

...

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '

  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: Th

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 66, column 4 to column 48)
  If this warning occurs sporadically, such as fo

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 0.005161 seconds
  1000 transitions using 10 leapfrog steps per transition would take 51.61 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropol

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_1u7b9tqj/model_fzarby6w.stan', line 66, column 4 to column 48)
  If this warning occurs sporadically, such as fo

In [19]:
with open('model_1.pkl', 'wb') as f:
    pickle.dump(posterior, f)    

In [20]:
with open('fit_1.pkl', 'wb') as g:
    pickle.dump(fit, g)         

In [21]:
#%%

# MODEL 2

model1 = """
data{
    int<lower=1> N1;
    array[N1] int CountsNr1d1_RC;
    array[N1] int CountsCry1_RC;
    array[N1] real jtime_RC;
    array[N1] real AreaNormed_RC;
    int<lower=1> N2;
    array[N2] int CountsBmal1_BC;
    array[N2] int CountsCry1_BC;
    array[N2] real jtime_BC;
    array[N2] real AreaNormed_BC;
    int <lower=1> N_f;
    array[N_f] real Nr1d1_params;
    array[N_f] real Cry1_params;
    array[N_f] real Bmal1_params;
    real w;
}
parameters{
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    freq_scaleCry1~ normal(0,100);
    freq_scaleNr1d1~ normal(0,100);
    freq_scaleBmal1~ normal(0,100);
    burstCry1~ normal(0,100);
    burstNr1d1~ normal(0,100);
    burstBmal1~ normal(0,100);
    beta_v_Cry1~ normal(0,100);
    beta_v_Nr1d1~ normal(0,100);
    beta_v_Bmal1~ normal(0,100);
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}    

"""

In [22]:
#%%

dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [23]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model1, data=dat, random_seed=194838)

# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=48, num_samples=2000, delta=0.95)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/job.11303346/httpstan_pc45eqs9/model_v3nn62d2.stan', line 79, column 27: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_pc45eqs9/model_v3nn62d2.stan', line 78, column 27: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_pc45eqs9/model_v3nn62d2.stan', line 77, column 26: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_pc45eqs9/model_v3nn62d2.stan', line 76, column 25: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in 

Sampling:  34% (48836/144000)
Sampling:  34% (48935/144000)
Sampling:  34% (49034/144000)
Sampling:  34% (49234/144000)
Sampling:  34% (49333/144000)
Sampling:  34% (49432/144000)
Sampling:  34% (49531/144000)
Sampling:  34% (49630/144000)
Sampling:  35% (49830/144000)
Sampling:  35% (49929/144000)
Sampling:  35% (50028/144000)
Sampling:  35% (50127/144000)
Sampling:  35% (50226/144000)
Sampling:  35% (50325/144000)
Sampling:  35% (50424/144000)
Sampling:  35% (50523/144000)
Sampling:  35% (50622/144000)
Sampling:  35% (50721/144000)
Sampling:  35% (50820/144000)
Sampling:  35% (50919/144000)
Sampling:  35% (51018/144000)
Sampling:  35% (51117/144000)
Sampling:  36% (51216/144000)
Sampling:  36% (51315/144000)
Sampling:  36% (51414/144000)
Sampling:  36% (51513/144000)
Sampling:  36% (51612/144000)
Sampling:  36% (51711/144000)
Sampling:  36% (51810/144000)
Sampling:  36% (51909/144000)
Sampling:  36% (52008/144000)
Sampling:  36% (52107/144000)
Sampling:  36% (52206/144000)
Sampling: 

Sampling:  56% (81000/144000)
Sampling:  56% (81100/144000)
Sampling:  56% (81200/144000)
Sampling:  56% (81300/144000)
Sampling:  57% (81400/144000)
Sampling:  57% (81500/144000)
Sampling:  57% (81600/144000)
Sampling:  57% (81700/144000)
Sampling:  57% (81800/144000)
Sampling:  57% (82000/144000)
Sampling:  57% (82100/144000)
Sampling:  57% (82200/144000)
Sampling:  57% (82300/144000)
Sampling:  57% (82400/144000)
Sampling:  57% (82500/144000)
Sampling:  57% (82600/144000)
Sampling:  57% (82700/144000)
Sampling:  58% (82800/144000)
Sampling:  58% (82900/144000)
Sampling:  58% (83000/144000)
Sampling:  58% (83100/144000)
Sampling:  58% (83200/144000)
Sampling:  58% (83300/144000)
Sampling:  58% (83500/144000)
Sampling:  58% (83600/144000)
Sampling:  58% (83700/144000)
Sampling:  58% (83800/144000)
Sampling:  58% (83900/144000)
Sampling:  58% (84000/144000)
Sampling:  58% (84100/144000)
Sampling:  58% (84200/144000)
Sampling:  59% (84300/144000)
Sampling:  59% (84400/144000)
Sampling: 

Sampling:  78% (112800/144000)
Sampling:  78% (112900/144000)
Sampling:  78% (113000/144000)
Sampling:  79% (113100/144000)
Sampling:  79% (113200/144000)
Sampling:  79% (113300/144000)
Sampling:  79% (113400/144000)
Sampling:  79% (113500/144000)
Sampling:  79% (113600/144000)
Sampling:  79% (113700/144000)
Sampling:  79% (113800/144000)
Sampling:  79% (113900/144000)
Sampling:  79% (114000/144000)
Sampling:  79% (114100/144000)
Sampling:  79% (114200/144000)
Sampling:  79% (114300/144000)
Sampling:  79% (114400/144000)
Sampling:  80% (114500/144000)
Sampling:  80% (114600/144000)
Sampling:  80% (114700/144000)
Sampling:  80% (114800/144000)
Sampling:  80% (114900/144000)
Sampling:  80% (115000/144000)
Sampling:  80% (115100/144000)
Sampling:  80% (115200/144000)
Sampling:  80% (115300/144000)
Sampling:  80% (115400/144000)
Sampling:  80% (115500/144000)
Sampling:  80% (115700/144000)
Sampling:  80% (115800/144000)
Sampling:  80% (115900/144000)
Sampling:  81% (116000/144000)
Sampling

...

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_ylcw4ry_/model_v3nn62d2.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_ylcw4ry_/model_v3nn62d2.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for 

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_ylcw4ry_/model_v3nn62d2.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp

  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_ylcw4ry_/model_v3nn62d2.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 0.008109 seconds
  1000 transitions using 10 leapfrog steps per transition would take 81.09 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_ylcw4ry_/model_v3nn62d2.stan', line 67, column 4 

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_ylcw4ry_/model_v3nn62d2.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 0.007918 seconds
  1000 transitions using 10 leapfrog steps per transition would take 79.18 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropol

  Gradient evaluation took 0.008181 seconds
  1000 transitions using 10 leapfrog steps per transition would take 81.81 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_ylcw4ry_/model_v3nn62d2.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/job.11303346/httpstan_ylcw4ry_/model_v3nn62d2.stan', line 67, column 4 to c

In [24]:
with open('model_2.pkl', 'wb') as f:
    pickle.dump(posterior, f)
    
with open('fit_2.pkl', 'wb') as g:
    pickle.dump(fit, g)

In [25]:
posterior = pickle.load(open('model_2.pkl', 'rb'))
fit = pickle.load(open('fit_2.pkl', 'rb')) 

a = fit

In [26]:
# load parameters from model M2
# freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=0)
# freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=0)
# freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=0)
# beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=0)
# beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=0)
# beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=0)
# burstCry1 = np.mean(a['burstCry1'],axis=0)
# burstNr1d1 = np.mean(a['burstNr1d1'],axis=0)
# burstBmal1 = np.mean(a['burstBmal1'],axis=0)

freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=1).squeeze()
freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=1).squeeze()
freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=1).squeeze()
beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=1).squeeze()
beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=1).squeeze()
beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=1).squeeze()
burstCry1 = np.mean(a['burstCry1'],axis=1).squeeze()
burstNr1d1 = np.mean(a['burstNr1d1'],axis=1).squeeze()
burstBmal1 = np.mean(a['burstBmal1'],axis=1).squeeze()

In [27]:
#%%
# MODEL 3 - phase noise

model = """
data{
    int<lower=1> N1;
    array[N1] int CountsNr1d1_RC;
    array[N1] int CountsCry1_RC;
    array[N1] real jtime_RC;
    array[N1] real AreaNormed_RC;
    int<lower=1> N2;
    array[N2] int CountsBmal1_BC;
    array[N2] int CountsCry1_BC;
    array[N2] real jtime_BC;
    array[N2] real AreaNormed_BC;
    real kappa;
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
    int <lower=1> N_f;
    array[N_f] real Nr1d1_params;
    array[N_f] real Cry1_params;
    array[N_f] real Bmal1_params;
    real w;
}
parameters{
    array[N1] real phi_i_add_RC;
    array[N2] real phi_i_add_BC;
    real<lower=0,upper=2> amp_scal;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4]-phi_i_add_RC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]-phi_i_add_RC[i]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+amp_scal*Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4]-phi_i_add_RC[i])+amp_scal*Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]-phi_i_add_RC[i]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4]-phi_i_add_BC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]-phi_i_add_BC[i]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+amp_scal*Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4]-phi_i_add_BC[i])+amp_scal*Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]-phi_i_add_BC[i]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    for (n in 1:N1) {
        phi_i_add_RC[n] ~ von_mises(0, kappa);
    }
    for (n in 1:N2) {
        phi_i_add_BC[n] ~ von_mises(0, kappa);
    }
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4]-phi_i_add_RC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]-phi_i_add_RC[i]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+amp_scal*Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4]-phi_i_add_RC[i])+amp_scal*Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]-phi_i_add_RC[i]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4]-phi_i_add_BC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]-phi_i_add_BC[i]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+amp_scal*Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4]-phi_i_add_BC[i])+amp_scal*Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]-phi_i_add_BC[i]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}        

"""

#%%

In [28]:
dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'kappa' : 2,
    'beta_v_Cry1' : beta_v_Cry1,
    'beta_v_Nr1d1' : beta_v_Nr1d1,
    'beta_v_Bmal1' : beta_v_Bmal1,
    'freq_scaleCry1' : freq_scaleCry1,
    'freq_scaleNr1d1' : freq_scaleNr1d1, 
    'freq_scaleBmal1' : freq_scaleBmal1, 
    'burstCry1' : burstCry1,
    'burstNr1d1' : burstNr1d1,
    'burstBmal1' : burstBmal1,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}
#%%

In [29]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model, data=dat, random_seed=194838)

# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=48, num_samples=2000, delta=0.95)

Building...



Building: found in cache, done.Messages from stanc:
    bound in its declaration. These hard constraints are not recommended, for
    two reasons: (a) Except when there are logical or physical constraints,
    it is very unusual for you to be sure that a parameter will fall inside a
    specified range, and (b) The infinite gradient induced by a hard
    constraint can cause difficulties for Stan's sampling algorithm. As a
    consequence, we recommend soft constraints rather than hard constraints;
    for example, instead of constraining an elasticity parameter to fall
    between 0, and 1, leave it unconstrained and give it a normal(0.5,0.5)
    prior distribution.
    is provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    is provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false 

Sampling:  13% (19000/144000)
Sampling:  13% (19100/144000)
Sampling:  13% (19200/144000)
Sampling:  13% (19300/144000)
Sampling:  13% (19400/144000)
Sampling:  14% (19500/144000)
Sampling:  14% (19600/144000)
Sampling:  14% (19700/144000)
Sampling:  14% (19800/144000)
Sampling:  14% (19900/144000)
Sampling:  14% (20000/144000)
Sampling:  14% (20100/144000)
Sampling:  14% (20200/144000)
Sampling:  14% (20300/144000)
Sampling:  14% (20400/144000)
Sampling:  14% (20500/144000)
Sampling:  14% (20600/144000)
Sampling:  14% (20700/144000)
Sampling:  14% (20800/144000)
Sampling:  15% (20900/144000)
Sampling:  15% (21000/144000)
Sampling:  15% (21100/144000)
Sampling:  15% (21200/144000)
Sampling:  15% (21300/144000)
Sampling:  15% (21400/144000)
Sampling:  15% (21500/144000)
Sampling:  15% (21600/144000)
Sampling:  15% (21700/144000)
Sampling:  15% (21800/144000)
Sampling:  15% (21900/144000)
Sampling:  15% (22000/144000)
Sampling:  15% (22100/144000)
Sampling:  15% (22200/144000)
Sampling: 

Sampling:  32% (45825/144000)
Sampling:  32% (45926/144000)
Sampling:  32% (46027/144000)
Sampling:  32% (46128/144000)
Sampling:  32% (46228/144000)
Sampling:  32% (46329/144000)
Sampling:  32% (46430/144000)
Sampling:  32% (46531/144000)
Sampling:  32% (46631/144000)
Sampling:  32% (46632/144000)
Sampling:  32% (46732/144000)
Sampling:  33% (46833/144000)
Sampling:  33% (46934/144000)
Sampling:  33% (46935/144000)
Sampling:  33% (47036/144000)
Sampling:  33% (47137/144000)
Sampling:  33% (47238/144000)
Sampling:  33% (47239/144000)
Sampling:  33% (47340/144000)
Sampling:  33% (47441/144000)
Sampling:  33% (47442/144000)
Sampling:  33% (47543/144000)
Sampling:  33% (47644/144000)
Sampling:  33% (47745/144000)
Sampling:  33% (47846/144000)
Sampling:  33% (47947/144000)
Sampling:  33% (48048/144000)
Sampling:  33% (48147/144000)
Sampling:  34% (48246/144000)
Sampling:  34% (48345/144000)
Sampling:  34% (48444/144000)
Sampling:  34% (48543/144000)
Sampling:  34% (48642/144000)
Sampling: 

Sampling:  50% (72700/144000)
Sampling:  51% (72800/144000)
Sampling:  51% (72900/144000)
Sampling:  51% (73000/144000)
Sampling:  51% (73100/144000)
Sampling:  51% (73200/144000)
Sampling:  51% (73300/144000)
Sampling:  51% (73400/144000)
Sampling:  51% (73500/144000)
Sampling:  51% (73600/144000)
Sampling:  51% (73700/144000)
Sampling:  51% (73800/144000)
Sampling:  51% (73900/144000)
Sampling:  51% (74000/144000)
Sampling:  51% (74100/144000)
Sampling:  52% (74200/144000)
Sampling:  52% (74300/144000)
Sampling:  52% (74400/144000)
Sampling:  52% (74500/144000)
Sampling:  52% (74600/144000)
Sampling:  52% (74700/144000)
Sampling:  52% (74800/144000)
Sampling:  52% (74900/144000)
Sampling:  52% (75000/144000)
Sampling:  52% (75100/144000)
Sampling:  52% (75200/144000)
Sampling:  52% (75300/144000)
Sampling:  52% (75400/144000)
Sampling:  52% (75500/144000)
Sampling:  52% (75600/144000)
Sampling:  53% (75700/144000)
Sampling:  53% (75800/144000)
Sampling:  53% (75900/144000)
Sampling: 

Sampling:  69% (100000/144000)
Sampling:  70% (100100/144000)
Sampling:  70% (100200/144000)
Sampling:  70% (100300/144000)
Sampling:  70% (100400/144000)
Sampling:  70% (100500/144000)
Sampling:  70% (100600/144000)
Sampling:  70% (100700/144000)
Sampling:  70% (100800/144000)
Sampling:  70% (100900/144000)
Sampling:  70% (101000/144000)
Sampling:  70% (101100/144000)
Sampling:  70% (101200/144000)
Sampling:  70% (101300/144000)
Sampling:  70% (101400/144000)
Sampling:  70% (101500/144000)
Sampling:  71% (101600/144000)
Sampling:  71% (101700/144000)
Sampling:  71% (101800/144000)
Sampling:  71% (101900/144000)
Sampling:  71% (102000/144000)
Sampling:  71% (102100/144000)
Sampling:  71% (102200/144000)
Sampling:  71% (102300/144000)
Sampling:  71% (102400/144000)
Sampling:  71% (102500/144000)
Sampling:  71% (102600/144000)
Sampling:  71% (102700/144000)
Sampling:  71% (102800/144000)
Sampling:  71% (102900/144000)
Sampling:  72% (103000/144000)
Sampling:  72% (103100/144000)
Sampling

Sampling:  88% (126400/144000)
Sampling:  88% (126500/144000)
Sampling:  88% (126600/144000)
Sampling:  88% (126700/144000)
Sampling:  88% (126800/144000)
Sampling:  88% (126900/144000)
Sampling:  88% (127000/144000)
Sampling:  88% (127100/144000)
Sampling:  88% (127200/144000)
Sampling:  88% (127300/144000)
Sampling:  88% (127400/144000)
Sampling:  89% (127500/144000)
Sampling:  89% (127600/144000)
Sampling:  89% (127700/144000)
Sampling:  89% (127800/144000)
Sampling:  89% (127900/144000)
Sampling:  89% (128000/144000)
Sampling:  89% (128100/144000)
Sampling:  89% (128200/144000)
Sampling:  89% (128300/144000)
Sampling:  89% (128400/144000)
Sampling:  89% (128500/144000)
Sampling:  89% (128600/144000)
Sampling:  89% (128700/144000)
Sampling:  89% (128800/144000)
Sampling:  90% (128900/144000)
Sampling:  90% (129000/144000)
Sampling:  90% (129100/144000)
Sampling:  90% (129200/144000)
Sampling:  90% (129300/144000)
Sampling:  90% (129400/144000)
Sampling:  90% (129500/144000)
Sampling

  Adjust your expectations accordingly!
  Gradient evaluation took 0.015925 seconds
  1000 transitions using 10 leapfrog steps per transition would take 159.25 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.015723 seconds
  1000 transitions using 10 leapfrog steps per transition would take 157.23 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.015697 seconds
  1000 transitions using 10 leapfrog steps per transition would take 156.97 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.015743 seconds
  1000 transitions using 10 leapfrog steps per transition would take 157.43 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.015732 seconds
  1000 transitions using 10 leapfrog steps per transition would take 157.32 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.015848 seconds
  1000 transitions using 10 leapfrog steps per transition would take 15

In [30]:
with open('model_3.pkl', 'wb') as f:
    pickle.dump(posterior, f)
    
with open('fit_3.pkl', 'wb') as g:
    pickle.dump(fit, g)

In [31]:
#%%     

# MODEL4

model1 = """
data{
    int<lower=1> N1;
    array[N1] int CountsNr1d1_RC;
    array[N1] int CountsCry1_RC;
    array[N1] real jtime_RC;
    array[N1] real AreaNormed_RC;
    int<lower=1> N2;
    array[N2] int CountsBmal1_BC;
    array[N2] int CountsCry1_BC;
    array[N2] real jtime_BC;
    array[N2] real AreaNormed_BC;
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    int <lower=1> N_f;
    array[N_f] real Nr1d1_params;
    array[N_f] real Cry1_params;
    array[N_f] real Bmal1_params;
    real w;
}
parameters{
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
    matrix[N1,2] eta_RC;
    matrix[N2,2] eta_BC;
    real<lower=0> stdevCry1;
    real<lower=0> stdevNr1d1; 
    real<lower=0> stdevBmal1; 
    cholesky_factor_corr[2] L_RC;
    cholesky_factor_corr[2] L_BC;
}
transformed parameters{
    matrix[2, 2] corr_RC;
    matrix[2, 2] cov_RC;
    vector[2] mu_vec_RC;
    vector[2] sigma_vec_RC;
    matrix[2, 2] corr_BC;
    matrix[2, 2] cov_BC;
    vector[2] mu_vec_BC;
    vector[2] sigma_vec_BC;
    corr_RC = L_RC*L_RC';
    corr_BC = L_BC*L_BC';
    mu_vec_RC[1] = burstCry1;
    mu_vec_RC[2] = burstNr1d1;
    mu_vec_BC[1] = burstCry1;
    mu_vec_BC[2] = burstBmal1;
    sigma_vec_RC[1] = stdevCry1;
    sigma_vec_RC[2] = stdevNr1d1;
    sigma_vec_BC[1] = stdevCry1;
    sigma_vec_BC[2] = stdevBmal1; 
    cov_RC = quad_form_diag(corr_RC, sigma_vec_RC);
    cov_BC = quad_form_diag(corr_BC, sigma_vec_BC);
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    vector[2] rescaled_eta;
    for ( i in 1:N1 ) {
        rescaled_eta = mu_vec_RC + diag_pre_multiply(sigma_vec_RC,L_RC)*(eta_RC[i,:]');
        b1 = exp(beta_v_Cry1*log(AreaNormed_RC[i])+rescaled_eta[1]);
        b2 = exp(beta_v_Nr1d1*log(AreaNormed_RC[i])+rescaled_eta[2]);
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        rescaled_eta = mu_vec_BC + diag_pre_multiply(sigma_vec_BC,L_BC)*(eta_BC[i,:]');
        b3 = exp(beta_v_Cry1*log(AreaNormed_BC[i])+rescaled_eta[1]);
        b4 = exp(beta_v_Bmal1*log(AreaNormed_BC[i])+rescaled_eta[2]);
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    to_vector(eta_RC) ~ normal(0, 1);
    to_vector(eta_BC) ~ normal(0, 1);
    burstCry1 ~ normal(0, 100);
    burstNr1d1 ~ normal(0, 100);
    burstBmal1 ~ normal(0, 100);
    L_RC ~ lkj_corr_cholesky(4.0);
    L_BC ~ lkj_corr_cholesky(4.0);
    stdevCry1 ~ normal(0, 100);
    stdevNr1d1 ~ normal(0, 100);
    stdevBmal1 ~ normal(0, 100);
}
generated quantities{ 
vector[N1] log_lik1;
vector[N1] log_lik2;
vector[N2] log_lik3;
vector[N2] log_lik4;
real mu1;
real mu2;
real mu3;
real mu4;
real r1;
real r2;
real r3;
real r4;
real b1;
real b2;
real b3;
real b4;
real f1;
real f2;
real f3;
real f4;
vector[2] rescaled_eta;
for ( i in 1:N1 ) {
    rescaled_eta = mu_vec_RC + diag_pre_multiply(sigma_vec_RC,L_RC)*(eta_RC[i,:]');
    b1 = exp(beta_v_Cry1*log(AreaNormed_RC[i])+rescaled_eta[1]);
    b2 = exp(beta_v_Nr1d1*log(AreaNormed_RC[i])+rescaled_eta[2]);
    f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
    f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
    mu1 = b1*f1;
    mu2 = b2*f2;
    r1 = f1;
    r2 = f2;
    log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
    log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
} 
for ( i in 1:N2 ) {
    rescaled_eta = mu_vec_BC + diag_pre_multiply(sigma_vec_BC,L_BC)*(eta_BC[i,:]');
    b3 = exp(beta_v_Cry1*log(AreaNormed_BC[i])+rescaled_eta[1]);
    b4 = exp(beta_v_Bmal1*log(AreaNormed_BC[i])+rescaled_eta[2]);
    f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
    f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
    mu3 = b3*f3;
    mu4 = b4*f4;
    r3 = f3;
    r4 = f4;
    log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
    log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
} 


}       

"""

In [32]:
# load parameters from model M2

posterior = pickle.load(open('model_2.pkl', 'rb'))
fit = pickle.load(open('fit_2.pkl', 'rb')) 

a = fit    

# freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=0)
# freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=0)
# freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=0)
# beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=0)
# beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=0)
# beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=0)
freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=1).squeeze()
freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=1).squeeze()
freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=1).squeeze()
beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=1).squeeze()
beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=1).squeeze()
beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=1).squeeze()

In [33]:
dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'beta_v_Cry1' : beta_v_Cry1,
    'beta_v_Nr1d1' : beta_v_Nr1d1,
    'beta_v_Bmal1' : beta_v_Bmal1,
    'freq_scaleCry1' : freq_scaleCry1,
    'freq_scaleNr1d1' : freq_scaleNr1d1, 
    'freq_scaleBmal1' : freq_scaleBmal1, 
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [34]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model1, data=dat, random_seed=194838)

# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=48, num_samples=2000, delta=0.95)

Building...

In file included from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/tbb/tbb_profiling.h:123,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/tbb/task.h:36,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/tbb/task_arena.h:23,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/prim/core/init_threadpool_tbb.hpp:18,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/prim/core.hpp:4,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/rev/core/Eigen_NumTraits.hpp:5,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/rev/core/typedefs.hpp:7,
                 from /s

In file included from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/prim/prob/normal_ccdf_log.hpp:5,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/prim/prob.hpp:243,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/prim.hpp:16,
                 from /scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/rev.hpp:16:
/scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/prim/prob/normal_lccdf.hpp: In function 'stan::return_type_t<T_x, T_sigma, T_l> stan::math::normal_lccdf(const T_y&, const T_loc&, const T_scale&)':
/scratch/user/shipalnomoo/.conda/envs/jl/lib/python3.11/site-packages/httpstan/include/stan/math/prim/prob/normal_lccdf.hpp:68: note: '-Wmisleading-indentation' is disabled from this point onwards, sin


Building: 141.0s, done.Messages from stanc:
Warning in '/tmp/job.11303346/httpstan_3023nkud/model_y7b6dgoj.stan', line 112, column 27: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_3023nkud/model_y7b6dgoj.stan', line 111, column 27: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_3023nkud/model_y7b6dgoj.stan', line 110, column 26: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/job.11303346/httpstan_3023nkud/model_y7b6dgoj.stan', line 107, column 27: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tm

Sampling:  13% (18500/144000)
Sampling:  13% (18600/144000)
Sampling:  13% (18700/144000)
Sampling:  13% (18800/144000)
Sampling:  13% (18900/144000)
Sampling:  13% (19000/144000)
Sampling:  13% (19100/144000)
Sampling:  13% (19200/144000)
Sampling:  13% (19300/144000)
Sampling:  13% (19400/144000)
Sampling:  14% (19500/144000)
Sampling:  14% (19600/144000)
Sampling:  14% (19700/144000)
Sampling:  14% (19800/144000)
Sampling:  14% (19900/144000)
Sampling:  14% (20000/144000)
Sampling:  14% (20100/144000)
Sampling:  14% (20200/144000)
Sampling:  14% (20300/144000)
Sampling:  14% (20400/144000)
Sampling:  14% (20500/144000)
Sampling:  14% (20600/144000)
Sampling:  14% (20700/144000)
Sampling:  14% (20800/144000)
Sampling:  15% (20900/144000)
Sampling:  15% (21000/144000)
Sampling:  15% (21100/144000)
Sampling:  15% (21200/144000)
Sampling:  15% (21300/144000)
Sampling:  15% (21400/144000)
Sampling:  15% (21500/144000)
Sampling:  15% (21600/144000)
Sampling:  15% (21700/144000)
Sampling: 

...

  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_g873uves/model_y7b6dgoj.stan', line 100, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_g873uves/model_y7b6dgoj.stan', line 100, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this w

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_g873uves/model_y7b6dgoj.stan', line 100, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_g873uves/model_y7b6dgoj.stan', line 100, column 4 to column 48)
  If this warning occurs sporadically, such a

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 0.015619 seconds
  1000 transitions using 10 leapfrog steps per transition would take 156.19 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_g873uves/model_y7b6dgoj.stan', line 102, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metr

  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_g873uves/model_y7b6dgoj.stan', line 100, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_g873uves/model_y7b6dgoj.stan', line 100, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this w

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/job.11303346/httpstan_g873uves/model_y7b6dgoj.stan', line 99, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


In [35]:
with open('model_4.pkl', 'wb') as f:
    pickle.dump(posterior, f)    

In [36]:
with open('fit_4.pkl', 'wb') as g:
    pickle.dump(fit, g)   
    
# a = fit.extract(permuted=True)